In [6]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import tensorflow as tf

ImportError: C extension: pandas.util.version not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

In [5]:
import os
import wave
import glob

In [11]:
from scipy.io import wavfile
from scipy import fft
from scipy import signal
#from matplotlib import cm
import IPython.display as ipd
#import matplotlib.pyplot as plt
from scipy.fft import fftshift
from scipy.spatial.distance import cdist
import librosa

In [4]:
#import neural models to help with detection 

from tensorflow import keras
from keras import layers
from keras import Input
from keras.layers import Conv1D, Conv2D, MaxPool1D, MaxPool2D, GlobalAvgPool1D, Dropout, BatchNormalization, Dense, LSTM, Flatten
# from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras.regularizers import l2
from keras import backend as K

## Final Project (Wake-word detection)

First, get the .wav files in the folder

In [13]:
# Specify the folder containing the .wav files
folder_path = '/Users/kerichen/Downloads/Animals/bird'

# Use glob to find all .wav files in the folder
bird_files = glob.glob(os.path.join(folder_path, '*.wav'))

# Iterate through the list of .wav files
for bird_file in bird_files:
    # Open the .wav file
    with wave.open(bird_file, 'rb') as wav_obj:
        # Extract and print some properties of the .wav file
       # print(f"Processing file: {wav_file}")
       # print(f"Number of channels: {wav_obj.getnchannels()}")
       # print(f"Sample width (bytes): {wav_obj.getsampwidth()}")
       # print(f"Frame rate (sample rate): {wav_obj.getframerate()}")
       # print(f"Number of frames: {wav_obj.getnframes()}")
       # print(f"Compression type: {wav_obj.getcomptype()}")
       # print(f"Compression name: {wav_obj.getcompname()}")
       # print("\n")

        print (bird_file)

/Users/kerichen/Downloads/Animals/bird/6a27a9bf_nohash_0.wav
/Users/kerichen/Downloads/Animals/bird/e77d88fc_nohash_1.wav
/Users/kerichen/Downloads/Animals/bird/d9d6559e_nohash_0.wav
/Users/kerichen/Downloads/Animals/bird/84bf12ff_nohash_0.wav
/Users/kerichen/Downloads/Animals/bird/ae04cdbe_nohash_0.wav
/Users/kerichen/Downloads/Animals/bird/00b01445_nohash_0.wav
/Users/kerichen/Downloads/Animals/bird/56f78b7e_nohash_0.wav
/Users/kerichen/Downloads/Animals/bird/b43c8f2f_nohash_0.wav
/Users/kerichen/Downloads/Animals/bird/6272b231_nohash_0.wav
/Users/kerichen/Downloads/Animals/bird/988e2f9a_nohash_1.wav
/Users/kerichen/Downloads/Animals/bird/d9b50b8b_nohash_2.wav
/Users/kerichen/Downloads/Animals/bird/8a28231e_nohash_0.wav
/Users/kerichen/Downloads/Animals/bird/92b2bf59_nohash_0.wav
/Users/kerichen/Downloads/Animals/bird/4cb874bb_nohash_2.wav
/Users/kerichen/Downloads/Animals/bird/881583a6_nohash_0.wav
/Users/kerichen/Downloads/Animals/bird/095847e4_nohash_0.wav
/Users/kerichen/Download

Next, we will extract the features from each of the folders with the phrases

In [14]:
librosa.load('/Users/kerichen/Downloads/Animals/bird/0a7c2a8d_nohash_0.wav')

(array([-0.00014037,  0.00117776, -0.00058977, ...,  0.00038463,
         0.00082914,  0.00069896], dtype=float32),
 22050)

In [15]:
def get_features(files):
    X = []
    spectrograms = []
    mel_spectrograms = []
    mfccs = []

    for file in files:
        wav, sr = librosa.load(file, sr=None)

        spectrogram = np.abs(librosa.stft(wav))

        mel_spectrogram = librosa.feature.melspectrogram(y=wav, sr=sr)

        mfcc = librosa.feature.mfcc(y=wav, sr=sr)

        X.append(wav)
        spectrograms.append(spectrogram)
        mel_spectrograms.append(mel_spectrogram)
        mfccs.append(mfcc)
        
    # Pad sequences to ensure consistent shape for batch processing
    max_length = max(map(len, X))
    X = np.array([np.pad(x, (0, max_length - len(x)), 'constant') for x in X])

    # Ensure spectrogram, mel spectrogram, and MFCC have the same shape
    spectrograms = np.array([np.pad(s, ((0, 0), (0, max_length - s.shape[1])), 'constant') for s in spectrograms])
    mel_spectrograms = np.array([np.pad(m, ((0, 0), (0, max_length - m.shape[1])), 'constant') for m in mel_spectrograms])
    mfccs = np.array([np.pad(m, ((0, 0), (0, max_length - m.shape[1])), 'constant') for m in mfccs])
    
    return X, spectrograms, mel_spectrograms, mfccs

In [16]:
x_data_bird, spectrograms_bird, mel_spectrograms_bird, mfcss_bird = get_features(bird_files)

In [17]:
D = librosa.amplitude_to_db(np.abs(librosa.stft(x_data_bird[0])), ref=np.max)


plt.figure(figsize = (10, 6))
librosa.display.specshow(D, y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram');

NameError: name 'plt' is not defined

In [18]:
# Specify the folder containing the .wav files
folder_path = '/Users/kerichen/Downloads/Animals/cat'

# Use glob to find all .wav files in the folder
cat_files = glob.glob(os.path.join(folder_path, '*.wav'))

# Iterate through the list of .wav files
#for cat_file in cat_files:
    # Open the .wav file
    #with wave.open(cat_file, 'rb') as wav_obj:
        # Extract and print some properties of the .wav file
       # print(f"Processing file: {wav_file}")
       # print(f"Number of channels: {wav_obj.getnchannels()}")
       # print(f"Sample width (bytes): {wav_obj.getsampwidth()}")
       # print(f"Frame rate (sample rate): {wav_obj.getframerate()}")
       # print(f"Number of frames: {wav_obj.getnframes()}")
       # print(f"Compression type: {wav_obj.getcomptype()}")
       # print(f"Compression name: {wav_obj.getcompname()}")
       # print("\n")

       #print (cat_file)

In [19]:
x_data_cat, spectrograms_cat, mel_spectrograms_cat, mfcss_cat = get_features(cat_files)

: 

In [14]:
# Specify the folder containing the .wav files
folder_path = '/Users/kerichen/Downloads/Animals/dog'

# Use glob to find all .wav files in the folder
dog_files = glob.glob(os.path.join(folder_path, '*.wav'))

# Iterate through the list of .wav files
#for dog_file in dog_files:
    # Open the .wav file
 #   with wave.open(dog_file, 'rb') as wav_obj:
        # Extract and print some properties of the .wav file
       # print(f"Processing file: {wav_file}")
       # print(f"Number of channels: {wav_obj.getnchannels()}")
       # print(f"Sample width (bytes): {wav_obj.getsampwidth()}")
       # print(f"Frame rate (sample rate): {wav_obj.getframerate()}")
       # print(f"Number of frames: {wav_obj.getnframes()}")
       # print(f"Compression type: {wav_obj.getcomptype()}")
       # print(f"Compression name: {wav_obj.getcompname()}")
       # print("\n")

      #  print (dog_file)

In [15]:
x_data_dog, spectrograms_dog, mel_spectrograms_dog, mfcss_dog = get_features(dog_files)

: 